In [2]:
import statsapi
import json
import sys
print(sys.version)


3.11.5 (main, Aug 25 2023, 13:19:53) [GCC 9.4.0]


# 본격적인 데이터 추출

In [ ]:
# 연도별 팀 id 저장
for year in range(1876, 2024):
    team_ids = []
    teams = statsapi.get("teams", {"season": year, "sportId":1}).get("teams")
    for team in teams:
        team_id = team["id"]
        team_name = team["name"]
        team_ids.append({"id": team_id, "name": team_name})
    with open(f"team_ids/team_ids{year}.json", "w") as f:        
        f.write(json.dumps(team_ids))

In [ ]:
# season 기반 team_standing 추출

# 1969년 이전은 division이 나눠져 있지 않아서 데이터를 따로 처리해야함.
for year in range(1969,2024):
    standings = statsapi.standings_data(
        leagueId="103,104",
        division="all",
        include_wildcard=True,
        season = year
    )
    filename = f"standing{year}.json"
    with open("teamstandings/"+filename, "a") as f:
        f.write(json.dumps(standings)+"\n")
    

In [ ]:
# season별 teamId 기반 team_stat 추출

for year in range(2023, 2024):
    with open(f"team_ids/team_ids{year}.json", "r") as f:
        teams = json.load(f)
        teams_stat = []
        for team in teams:
            team_id = team["id"]
            teams_stat.append(statsapi.get('team_stats', {'teamId':team_id,'season':year,'group':'hitting,pitching,fielding','stats':'season'}))
            with open(f"teamstats/team_stat{year}.json", "w") as f:
                f.write(json.dumps(teams_stat))


In [2]:
# playerId 수집
import statsapi
import json
players = statsapi.get("sports_players",{"sportId":1,"season":2023})
with open(f"players_id/players_id2023.json", "w") as f:
        f.write(json.dumps(players))

In [ ]:
# matchId 수집
from datetime import date

for year in range(1901,2024):
    start = date(year,1,1)
    end = date(year,12,31)

    matches = statsapi.schedule(
        start_date=start,
        end_date=end,
        sportId=1
    )

    match_ids = []
    for match in matches:
        game_id = match["game_id"]
        summary = match["summary"]
        match_ids.append({"game_id":game_id,"summary":summary})
    with open(f"match_ids/match_ids{year}.json","w") as f:
        f.write(json.dumps(match_ids))


In [ ]:
# matchId별 linescore 수집
import json
def get_linescores(start, end):
    for year in range(start, end):
        with open(f"match_ids/match_ids{year}.json", "r") as f:
            match_ids = json.load(f)
        
        linescores = []

        for match_info in match_ids:
            game_id = match_info.get("game_id", None)
            game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
            linescores.append({"game_id":game_id,"linescore":game_linescore})
        with open(f"linescores/linescores{year}.json", "w") as f2:
            f2.write(json.dumps(linescores))

In [3]:
# season별 선수 ID 수집
import json

for year in range(2023, 2024):
    players_info = statsapi.get("sports_players",{"sportId":1,"season":year}).get("people")
    players_id = []
    for player_info in players_info:
        player_id = player_info.get("id",None)
        player_name = player_info.get("fullName",None)
        players_id.append({"id":player_id, "fullName":player_name})
    with open(f"players_id/players_id{year}.json", "w") as f:
        f.write(json.dumps(players_id))

In [ ]:
#중복되는 player id 제거하여 json으로 저장
import pandas as pd
import json
all_players_id = pd.DataFrame()
for year in range(1876,2024):
    with open(f"players_id/players_id{year}.json", "r") as f:
        players_id = json.load(f)

    players_id_df = pd.DataFrame(players_id)

    all_players_id = pd.concat([all_players_id, players_id_df])
all_players_id = all_players_id.sort_values("id")
all_players_id = all_players_id.reset_index(drop=True)
print(all_players_id)

all_players_id = all_players_id.drop_duplicates()
all_players_id = all_players_id.reset_index(drop=True)
print(all_players_id)

all_players_id.to_json("all_players.json", orient="records")

In [5]:
def player_stat_data(
    personId, group, type, sportId
):
    """Returns a list of current season or career stat data for a given player."""
    params = {
        "personId": personId,
        "hydrate": "stats(group="
        + group
        + ",type="
        + type
        + ",sportId="
        + str(sportId)
        + ")",
    }
    r = statsapi.get("person", params)

    stat_groups = []

    player = r["people"][0]

    return player

In [7]:
# player__id별
import statsapi
import json

players_stat = []
with open("all_players.json", "r") as f2:
    players_stat = json.load(f2)
for player in players_stat:
    player_id = player["id"]
    player_stat = player_stat_data(player_id, group="[hitting,pitching,fielding]", type="[career,yearByYear]", sportId=1)
    with open(f"players_stat/players_stat_{player_id}.json", "w") as f:
        f.write(json.dumps(player_stat))

In [2]:
# 필요한 데이터 정의
def player_stat_data(
    personId, group, type, sportId
):
    """Returns a list of current season or career stat data for a given player."""
    params = {
        "personId": personId,
        "hydrate": "stats(group="
        + group
        + ",type="
        + type
        + ",sportId="
        + str(sportId)
        + ")",
    }
    r = statsapi.get("person", params)
    player = r["people"][0]

    return player

In [4]:
#2023 player__id별
import statsapi
import json

with open("players_id/players_id2023.json", "r") as f2:
    player_ids = json.load(f2)
for p in player_ids:
    p_id = p.get("id")
    player_stat = player_stat_data(p_id, group="[hitting,pitching,fielding]", type="[career,yearByYear]", sportId=1)
    with open(f"players_stat/players_stat_{p_id}.json", "w") as f:
        f.write(json.dumps(player_stat))

In [ ]:
#boxscore collect
import statsapi
import json

for year in range(2023, 1901,-1):
    match_ids = []
    with open(f"match_ids/match_ids{year}.json", "r") as f:
        match_ids = json.load(f)
    
    for match in match_ids:
        game_id = match["game_id"]
        boxscore = statsapi.boxscore_data(game_id, timecode=None)
        input ={"game_id":game_id, "boxscore": boxscore}
        with open(f"boxscores/boxscore_{game_id}.json", "w") as f:
            f.write(json.dumps(input))